## Initial Setup
*Setup input and output object stores
*Get execution role to have security access to AWS account

In [2]:
from sagemaker import get_execution_role

#Bucket location to save custom code
custom_code_upload_location = 's3://kb-sagemaker/customcode/mxnet'

#Bucket location where results of model training are saved.
model_artifacts_location = 's3://kb-sagemaker/artifacts'

#IAM execution role that gives Amazon SageMaker access to resources in your AWS account.
#We can use the Amazon SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

# Setup Estimator
Estimator server for training

In [3]:
from sagemaker.mxnet import MXNet

mnist_estimator = MXNet(entry_point='/home/ec2-user/sample-notebooks/sagemaker-python-sdk/mxnet_mnist/mnist.py',
                        role=role,
                        output_path=model_artifacts_location,
                        code_location=custom_code_upload_location,
                        train_instance_count=1, 
                        train_instance_type='ml.m4.xlarge',
                        hyperparameters={'learning_rate': 0.1})

# Train Model
Load test data an train model deployed from previous step

In [4]:
%%time
import boto3

region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/train'.format(region)
test_data_location = 's3://sagemaker-sample-data-{}/mxnet/mnist/test'.format(region)

mnist_estimator.fit({'train': train_data_location, 'test': test_data_location})

INFO:sagemaker:Creating training-job with name: sagemaker-mxnet-2018-04-12-21-02-24-757


.................................................
2018-04-12 21:06:20,250 INFO - root - running container entrypoint
2018-04-12 21:06:20,250 INFO - root - starting train task
2018-04-12 21:06:20,256 INFO - container_support.training - Training starting
2018-04-12 21:06:22,417 INFO - mxnet_container.train - MXNetTrainingEnvironment: {'enable_cloudwatch_metrics': False, 'available_gpus': 0, 'channels': {u'test': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}, u'train': {u'TrainingInputMode': u'File', u'RecordWrapperType': u'None', u'S3DistributionType': u'FullyReplicated'}}, '_ps_verbose': 0, 'resource_config': {u'current_host': u'algo-1', u'hosts': [u'algo-1']}, 'user_script_name': u'mnist.py', 'input_config_dir': '/opt/ml/input/config', 'channel_dirs': {u'test': u'/opt/ml/input/data/test', u'train': u'/opt/ml/input/data/train'}, 'code_dir': '/opt/ml/code', 'output_data_dir': '/opt/ml/output/data/', 'output_dir': '/opt/ml/output

2018-04-12 21:06:38,170 INFO - root - Epoch[9] Batch [200]#011Speed: 44914.54 samples/sec#011accuracy=0.974800
2018-04-12 21:06:38,409 INFO - root - Epoch[9] Batch [300]#011Speed: 41918.73 samples/sec#011accuracy=0.978500
2018-04-12 21:06:38,632 INFO - root - Epoch[9] Batch [400]#011Speed: 44870.82 samples/sec#011accuracy=0.977000
2018-04-12 21:06:38,850 INFO - root - Epoch[9] Batch [500]#011Speed: 45907.97 samples/sec#011accuracy=0.977000
2018-04-12 21:06:39,066 INFO - root - Epoch[9] Train-accuracy=0.976465
2018-04-12 21:06:39,066 INFO - root - Epoch[9] Time cost=1.345
2018-04-12 21:06:39,173 INFO - root - Epoch[9] Validation-accuracy=0.969800
2018-04-12 21:06:39,419 INFO - root - Epoch[10] Batch [100]#011Speed: 41081.44 samples/sec#011accuracy=0.979604
2018-04-12 21:06:39,664 INFO - root - Epoch[10] Batch [200]#011Speed: 40857.02 samples/sec#011accuracy=0.977500
2018-04-12 21:06:39,887 INFO - root - Epoch[10] Batch [300]#011Speed: 44866.59 samples/sec#011accuracy=0.980400
2018-04-12

2018-04-12 21:06:54,778 INFO - root - Epoch[20] Batch [100]#011Speed: 41812.33 samples/sec#011accuracy=0.996139
2018-04-12 21:06:55,015 INFO - root - Epoch[20] Batch [200]#011Speed: 42288.84 samples/sec#011accuracy=0.994700
2018-04-12 21:06:55,250 INFO - root - Epoch[20] Batch [300]#011Speed: 42555.93 samples/sec#011accuracy=0.995300
2018-04-12 21:06:55,504 INFO - root - Epoch[20] Batch [400]#011Speed: 39435.75 samples/sec#011accuracy=0.995300
2018-04-12 21:06:55,735 INFO - root - Epoch[20] Batch [500]#011Speed: 43353.10 samples/sec#011accuracy=0.996600
2018-04-12 21:06:56,003 INFO - root - Epoch[20] Train-accuracy=0.995354
2018-04-12 21:06:56,004 INFO - root - Epoch[20] Time cost=1.466
2018-04-12 21:06:56,108 INFO - root - Epoch[20] Validation-accuracy=0.976200
2018-04-12 21:06:56,346 INFO - root - Epoch[21] Batch [100]#011Speed: 42370.90 samples/sec#011accuracy=0.996733
2018-04-12 21:06:56,560 INFO - root - Epoch[21] Batch [200]#011Speed: 46604.15 samples/sec#011accuracy=0.996300
201

# Deploy Trained Model to Endpoint for Predictions
Create server instance to host trained model and act as an endpoint for prediction API calls

In [5]:
%%time

predictor = mnist_estimator.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-mxnet-2018-04-12-21-02-24-757
INFO:sagemaker:Creating endpoint with name sagemaker-mxnet-2018-04-12-21-02-24-757


--------------------------------------------------------------------------!CPU times: user 304 ms, sys: 24 ms, total: 328 ms
Wall time: 6min 19s


# Setup HTML Drawing Client

In [47]:
from IPython.display import HTML
HTML(open("/home/ec2-user/sample-notebooks/sagemaker-python-sdk/mxnet_mnist/input.html").read())

# Make inference for above drawing

In [58]:
response = predictor.predict(data)
print('Raw prediction result:')
print(response)

labeled_predictions = list(zip(range(10), response[0]))
print('Labeled predictions: ')
print(labeled_predictions)

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print('Most likely answer: {}'.format(labeled_predictions[0]))

Raw prediction result:
[[1.162921433548467e-14, 0.0004702737496700138, 0.9995296001434326, 6.82744314417505e-08, 5.473607373287278e-16, 9.93337665980865e-16, 1.7941066384266624e-12, 2.0634491448845438e-08, 9.843352799521199e-09, 2.5989752216571644e-18]]
Labeled predictions: 
[(0, 1.162921433548467e-14), (1, 0.0004702737496700138), (2, 0.9995296001434326), (3, 6.82744314417505e-08), (4, 5.473607373287278e-16), (5, 9.93337665980865e-16), (6, 1.7941066384266624e-12), (7, 2.0634491448845438e-08), (8, 9.843352799521199e-09), (9, 2.5989752216571644e-18)]
Most likely answer: (2, 0.9995296001434326)


In [59]:
print(data)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 